In [ ]:
#In this notebook,we  are going use many time Series Techniques,ARIMA Model(Auto regression),ARIMAX Model
#Auto Arima and analysing trends

# Contents
 
*  Importing Liabraries
*  Exploring and Modifying data
*  Sales Predictions
*  Models Implimentation
*  Analysing Trends

In [ ]:
#importing Liabraries
import numpy as np 
import pandas as pd 
import plotly.express as px
import plotly.graph_objs as go
import plotly as py
from plotly import tools
from plotly.offline import iplot
import matplotlib.pyplot as plt
from matplotlib import dates
import seaborn as sns
from math import sqrt
import statsmodels.api as sm
#from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools  import adfuller
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acovf, acf
from statsmodels.graphics.tsaplots import plot_acf
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):

    for filename in filenames:
        print(os.path.join(dirname, filename))
        
#Graph Settings
plt.rcParams['figure.figsize'] = (15,4)
plt.rcParams['axes.grid'] = True
plt.rcParams['font.size'] = 14

In [ ]:
#Importing Packages
!pip install pmdarima
from pmdarima import auto_arima

In [ ]:
#Loading Data
dt = pd.read_csv("../input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv", parse_dates=["created_at", "Working Date"], low_memory=False)

In [ ]:
#Basic EDA
dt.info()

In [ ]:
print("Data Dimensions are: ", dt.shape)
print("Columns: ", dt.columns)

In [ ]:
dt.rename(columns = {' MV ':'MV'}, inplace = True)
#Droping Null Columns
dt = dt.iloc[:, :-5]
dt = dt.dropna(how = 'all') 
dt.columns

In [ ]:
#Casting on Data types
dt['Customer ID'] = dt['Customer ID'].astype(str)
dt['item_id'] = dt['item_id'].astype(str)
dt['qty_ordered'] = dt['qty_ordered'].astype(int)  
dt['Year'] = dt['Year'].astype(int)  
dt['Month'] = dt['Month'].astype(int) 

In [ ]:
#Data Summary
dt.describe()

In [ ]:
dt = dt.sort_values('created_at')
#Adding New Features
dtg = dt.groupby('created_at')['grand_total'].sum().reset_index()
dtq = dt.groupby('created_at')['qty_ordered'].sum().reset_index()
dtd = dt.groupby('created_at')['discount_amount'].sum().reset_index()
#Comput count for non numeric values
dts = dt.groupby('created_at')['sku'].count().reset_index() 
dtst = dt.groupby('created_at')['status'].count().reset_index()
# new data set
p = pd.DataFrame(dtg) 
p['qty_ordered'] = dtq['qty_ordered']
p['discount_amount'] = dtd['discount_amount']
p['sku'] = dts['sku']
p['status'] = dtst['status']
#Cumulative Sum
p['cum_grand_total'] = p['grand_total'].cumsum()
p['cum_qty_ordered'] = p['qty_ordered'].cumsum()
p['cum_discount_amount'] = p['discount_amount'].cumsum()
p['cum_sku_cnt'] = p['sku'].cumsum()
p['cum_status_cnt'] = p['status'].cumsum()
# Date features
p['Dateofmonth'] = p['created_at'].dt.day
p['Month'] = p['created_at'].dt.month
p['Week'] = p['created_at'].dt.week
p['Dayofweek'] = p['created_at'].dt.dayofweek # 0 = monday.
p['Weekdayflg'] = (p['Dayofweek'] // 5 != 1).astype(float)
p['Month'] = p['created_at'].dt.month
p['Quarter'] = p['created_at'].dt.quarter
p['Dayofyear'] = p['created_at'].dt.dayofyear

In [ ]:
p.head()

## **Creating a Sub-Dataset for Sales Predictions**

In [ ]:
#selecting first two column
p = p.iloc[:, 0:2]  
p.head()

In [ ]:
#Converting this column to datetime
p['created_at'] = pd.to_datetime(p['created_at'],format='%Y-%m-%d')

In [ ]:
#Accessing any specific date
p.loc[1, 'created_at'].day_name()
#We can also convert dates into days
p['created_at'].dt.day_name()

In [ ]:
p.head()

In [ ]:
#For Time Series,Date column must be in indexes
p = p.set_index('created_at')
from datetime import datetime
p.head()

In [ ]:
#Plotting graph between year and Total sales
sns.lineplot(x = "created_at" , y = "grand_total",data= p )
plt.show()

In [ ]:
#Checking Dataset is stationary or not
results = adfuller(p['grand_total'])
print(results)

#### Here 0th value is (-6.75) which shows test Statistic (More negative it is more stationary dataset will be)
#### Here 1th value is (2.82 e-09) which shows P-value (More negative p-value in data is considered as more stationary)
## Following conditions must be fulfilled for making dataset stationary
1. Test Statistics needed to be below -2.91 
2. Test Statistics has less value than critical value at 5%
3.  P-value is less than 0.5

In [ ]:
#We can also print p-value,Test statistics and Critical values as follows
results = adfuller(p['grand_total'])
print('Test Statistics: %f' % results[0])
print('p-value: %f' % results[1])
print('Critical Values: ')
for key, value in results[4].items():
    print('\t%s: %.3f' % (key,value))

## As dataset is Stationary but we can improve its stationarity by using following techniques
1. Difference with diff()
2. Log
3. Substracting previous value with Shift()
4. Differencing Simple Moving Average
5. Seasonal Decomposition

In [ ]:
#1.Difference with diff()
df_stationary = p.diff().dropna()
result = adfuller(df_stationary)
print(result)

## 2.Log

In [ ]:
indexeddataset_logscale = np.log(p['grand_total'])
result = adfuller(indexeddataset_logscale)
print(result)

## 3.Substracting previous value with Shift()

In [ ]:
datasetLogDiffShifting = indexeddataset_logscale - indexeddataset_logscale.shift()
datasetLogDiffShifting.dropna(inplace=True)
result = adfuller(datasetLogDiffShifting)
print(result)

## 4.Differencing Simple Moving Average

In [ ]:
movingaverage = indexeddataset_logscale.rolling(window=12).mean()
movingstd = indexeddataset_logscale.rolling(window=12).std()
datasetlogscaleminusmovingaverage = indexeddataset_logscale - movingaverage
datasetlogscaleminusmovingaverage.dropna(inplace=True)
result = adfuller(datasetlogscaleminusmovingaverage)
print(result)

## Differencing Simple Moving Average Provides us most suitable P-value and Test statistics

In [ ]:
plot = plt.plot(datasetlogscaleminusmovingaverage,color='blue',label='Original')
mean = plt.plot(movingaverage, color='red',label='Rolling Mean')
std = plt.plot(movingstd, color='Black',label='Rolling STD')
plt.legend(loc='best')
#plt.title()
plt.show(block=True)

## 5.Seasonal Decomposition
##### By using this,we can find Seasonality,trends and residuals.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(datasetlogscaleminusmovingaverage)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
plt.figure(figsize=(12,7))
plt.subplot(411)
plt.plot(datasetlogscaleminusmovingaverage, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend,label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual,label='Residuals')
plt.legend(loc='best')

In [ ]:
datasetlogscaleminusmovingaverage.head()

In [ ]:
type(datasetlogscaleminusmovingaverage)

In [ ]:
#Converting above from Series to dataframe
datasetlogscaleminusmovingaverage = pd.Series(datasetlogscaleminusmovingaverage)
new_dataset = datasetlogscaleminusmovingaverage.to_frame()
new_dataset.head()

In [ ]:
type(new_dataset)

# Applying ARIMA Model(Auto Regression)

In [ ]:
#Autocorrelation
ecom_df = new_dataset
acf(ecom_df, fft=False)
sm.graphics.tsa.plot_acf(ecom_df.values.squeeze(), lags=60);

In [ ]:
sm.graphics.tsa.plot_pacf(ecom_df.values.squeeze(), lags=60,
                          method='ywunbiased');

sm.graphics.tsa.plot_pacf(ecom_df.values.squeeze(), lags=60,
                         method='ols');

In [ ]:
# Change frequency to day
ecom_df = new_dataset.asfreq('d')

# Set style for seaborn plot
sns.set_style('darkgrid')

# Add automatic datetime converters
pd.plotting.register_matplotlib_converters()

# Default figure size
sns.mpl.rc('figure',figsize=(12, 6))

# Plot daily max temps
fig, ax = plt.subplots()
ax = ecom_df.plot(ax=ax)

# Creating model 
model = AutoReg(ecom_df['grand_total'],30)
model_fit = model.fit()

# Define training and testing area
len(ecom_df) # 778 observations
train_df = ecom_df.iloc[:620] 
test_df = ecom_df.iloc[620:] 

# Define training model for 159 days 
train_model = AutoReg(ecom_df['grand_total'], 159).fit(cov_type="HC0")

# Define start and end for prediction 
start = len(train_df)
end = len(train_df) + len(test_df) - 1
prediction = train_model.predict(start=start, end=end, dynamic=True)
# Plot testing data with prediction
ax = test_df.plot(ax=ax) # Orange
ax = prediction.plot(ax=ax) # Green

# Predicting Sales on Daily Time frame

In [ ]:
sns.mpl.rc('figure',figsize=(12, 6))
fig, ax = plt.subplots()
ax = ecom_df.plot(ax=ax)
model = AutoReg(ecom_df['grand_total'], 30)
model_fit = model.fit()
len(ecom_df) # 778 observations
train_df = ecom_df.iloc[:620] # 80%
test_df = ecom_df.iloc[620:] # Last 20%
# Define training model for 159 days 
train_model = AutoReg(ecom_df['grand_total'], 159).fit(cov_type="HC0")
start = len(train_df)
end = len(train_df) + len(test_df) - 1
prediction = train_model.predict(start=start, end=end, dynamic=True)
ax = test_df.plot(ax=ax) 
ax = prediction.plot(ax=ax) 

# Predict 100 days into the future
forecast = train_model.predict(start=end, end=end+100, dynamic=True)
forecast.plot(ax=ax) 

**In Above Graph, Different line Colours are used**
1. Red line shows predictions
2. Green Line shows training part of Dataset
3. Orange Line shows Test part of Dataset

In [ ]:
mean_absolute_error(test_df, prediction)

In [ ]:
ecom_df = new_dataset.asfreq('m')
ecom_df.head()

# Predicting Sales on Monthly Time frame

In [ ]:
# Set style for seaborn plot
sns.set_style('darkgrid')

# Add automatic datetime converters
pd.plotting.register_matplotlib_converters()

# Default figure size
sns.mpl.rc('figure',figsize=(12, 6))

# Plot monthly max temps
fig, ax = plt.subplots()
ax = ecom_df.plot(ax=ax)

# Creating model 
model = AutoReg(ecom_df['grand_total'],10)
model_fit = model.fit()

# Define training and testing area
len(ecom_df) # 25 observations
train_df = ecom_df.iloc[:16] 
test_df = ecom_df.iloc[16:] 

# Define training model for 5 days 
train_model = AutoReg(ecom_df['grand_total'], 5).fit(cov_type="HC0")

# Define start and end for prediction 
start = len(train_df)
end = len(train_df) + len(test_df) - 1
prediction = train_model.predict(start=start, end=end, dynamic=True)

# Predict 4 months into the future
forecast = train_model.predict(start=end, end=end+4, dynamic=True)
forecast.plot(ax=ax)

In [ ]:
mean_absolute_error(test_df, prediction)

# Applying SARIMAX Model

In [ ]:
new_dataset = datasetlogscaleminusmovingaverage.to_frame()
ecom_df = new_dataset.asfreq('d')
ecom_df.head()

In [ ]:
ecom_df.plot(figsize=(16,8))
# We verify that there is a seasonal component
res = seasonal_decompose(ecom_df, model='add')
res.plot()

In [ ]:
#Auto Arima
auto_arima(ecom_df, seasonal=True, m=12, trace=True).summary()

In [ ]:
train_df = ecom_df.iloc[:583] # 80%
test_df = ecom_df.iloc[583:] # 20%
model = SARIMAX(train_df, order=(0, 1, 3),
               seasonal_order=(1, 0, 1, 12))
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) - 1
forecast = res.predict(start, end )
forecast = pd.DataFrame(forecast,index = test_df.index,columns=['Prediction'])
plt.plot(train_df, label='Train')
plt.plot(test_df, label='Test')
plt.plot(forecast, label='Prediction')
plt.show()

In [ ]:
# Predict the future
model = SARIMAX(ecom_df, order=(0, 1, 3),
               seasonal_order=(1, 0, 1, 12))
res = model.fit()
future = res.predict(len(ecom_df), len(ecom_df)+12, 
                    typ='levels').rename('Future')
test_df.plot(legend=True, figsize=(16,8))
future.plot(legend=True)

### This model works slightly better than Above model

# TRENDS

### Lets find out overall trend

In [ ]:
dt = dt.sort_values('created_at')
#Adding New Features
dtg = dt.groupby('created_at')['grand_total'].sum().reset_index()
dtq = dt.groupby('created_at')['qty_ordered'].sum().reset_index()
dtd = dt.groupby('created_at')['discount_amount'].sum().reset_index()
#Comput count for non numeric values
dts = dt.groupby('created_at')['sku'].count().reset_index() 
dtst = dt.groupby('created_at')['status'].count().reset_index()
# new data set
p = pd.DataFrame(dtg) 
p['qty_ordered'] = dtq['qty_ordered']
p['discount_amount'] = dtd['discount_amount']
p['sku'] = dts['sku']
p['status'] = dtst['status']
#Cumulative Sum
p['cum_grand_total'] = p['grand_total'].cumsum()
p['cum_qty_ordered'] = p['qty_ordered'].cumsum()
p['cum_discount_amount'] = p['discount_amount'].cumsum()
p['cum_sku_cnt'] = p['sku'].cumsum()
p['cum_status_cnt'] = p['status'].cumsum()
# Date features
p['Dateofmonth'] = p['created_at'].dt.day
p['Month'] = p['created_at'].dt.month
p['Week'] = p['created_at'].dt.week
p['Dayofweek'] = p['created_at'].dt.dayofweek # 0 = monday.
p['Weekdayflg'] = (p['Dayofweek'] // 5 != 1).astype(float)
p['Month'] = p['created_at'].dt.month
p['Quarter'] = p['created_at'].dt.quarter
p['Dayofyear'] = p['created_at'].dt.dayofyear

p.head()

In [ ]:
trend_df = p.iloc[:, 0:2]  
trend_df.head()

In [ ]:
#Creating another column
trend_df['Year_month'] = trend_df['created_at'].apply(lambda x: x.strftime('%Y-%m'))
trend_df.head()

In [ ]:
#Grouping Month Year
trend_df = trend_df.groupby('Year_month').sum()['grand_total'].reset_index()

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(trend_df['Year_month'],trend_df['grand_total'],color='#b80045')
plt.xticks(rotation='vertical',size=8)
plt.show()

### We observe overall trend from july-2016 to aug-2018 and concluded following points
* #### Purchases increases in last Three months ( October,November and December ) of 2016 and 2017
* #### Most e-commerce buyers purchases products in November
* #### November is most suitable month for sellers to market their products
* #### Overall volume of ecommerce sales is increasing year by year 
* #### For example,you can compare first six months of 2017 with 2018

In [ ]:
# If you like my notebook,don,t forget to upvote.